<a href="https://colab.research.google.com/github/DeeeTeeee/SepsisClassificationWithFastAPI/blob/main/Sepsis_ClassificationEDA_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
#!pip install pandas-profiling
#!pip install gradio

In [135]:
# import libraries**
import pandas as pd
import numpy as np
#from pandas_profiling import ProfileReport
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score

In [136]:
url_test = 'https://raw.githubusercontent.com/DeeeTeeee/SepsisClassificationWithFastAPI/main/Datasets/Paitients_Files_Test.csv'
test_df = pd.read_csv(url_test,index_col=0)
#df = pd.read_csv(url)
url_train = 'https://raw.githubusercontent.com/DeeeTeeee/SepsisClassificationWithFastAPI/main/Datasets/Paitients_Files_Train.csv'

train_df = pd.read_csv(url_train,index_col=0)
print(test_df.head(5))
print(train_df.head(5))

           PRG   PL  PR  SK   TS   M11    BD2  Age  Insurance
ID                                                           
ICU200609    1  109  38  18  120  23.1  0.407   26          1
ICU200610    1  108  88  19    0  27.1  0.400   24          1
ICU200611    6   96   0   0    0  23.7  0.190   28          1
ICU200612    1  124  74  36    0  27.8  0.100   30          1
ICU200613    7  150  78  29  126  35.2  0.692   54          0
           PRG   PL  PR  SK   TS   M11    BD2  Age  Insurance   Sepssis
ID                                                                     
ICU200010    6  148  72  35    0  33.6  0.627   50          0  Positive
ICU200011    1   85  66  29    0  26.6  0.351   31          0  Negative
ICU200012    8  183  64   0    0  23.3  0.672   32          1  Positive
ICU200013    1   89  66  23   94  28.1  0.167   21          1  Negative
ICU200014    0  137  40  35  168  43.1  2.288   33          1  Positive


I will like to change the column names so I can understand them at one glance

In [137]:
# Define a dictionary with the new column names
new_column_names = {
    'PRG': 'Plasma glucose',
    'PL': 'Blood Work Result-1',
    'PR': 'Blood Pressure',
    'SK': 'Blood Work Result-2',
    'TS': 'Blood Work Result-3',
    'M11': 'Body mass index',
    'BD2': 'Blood Work Result-4'
}

# Rename the columns in train_df and test_df using the dictionary
train_df.rename(columns=new_column_names, inplace=True)
test_df.rename(columns=new_column_names, inplace=True)

In [138]:
train_df.sample(5)

,Plasma glucose,Blood Work Result-1,Blood Pressure,Blood Work Result-2,Blood Work Result-3,Body mass index,Blood Work Result-4,Age,Insurance,Sepssis
ID,,,,,,,,,,
ICU200135,1,88,30,42,99,55.0,0.496,26,1,Positive
ICU200190,6,87,80,0,0,23.2,0.084,32,1,Negative
ICU200049,4,111,72,47,207,37.1,1.390,56,0,Positive
ICU200213,2,99,70,16,44,20.4,0.235,27,1,Negative
ICU200068,0,146,82,0,0,40.5,1.781,44,1,Negative


In [139]:
print(train_df.shape, test_df.shape)

(599, 10) (169, 9)


In [140]:
train_df.columns

Index(['Plasma glucose', 'Blood Work Result-1', 'Blood Pressure',
       'Blood Work Result-2', 'Blood Work Result-3', 'Body mass index',
       'Blood Work Result-4', 'Age', 'Insurance', 'Sepssis'],
      dtype='object')

In [91]:
# # Create a pandas profiling report for train_df
# train_profile = ProfileReport(train_df, title='Train Dataset Pandas Profiling Report', explorative=True)

# # Display the pandas profiling report for train_df in Colab
# display(train_profile)

# # Create a pandas profiling report for test_df
# test_profile = ProfileReport(test_df, title='Test Dataset Pandas Profiling Report', explorative=True)

# # Display the pandas profiling report for test_df in Colab
# display(test_profile)

# Modeling

In [141]:
# Define a dictionary to map labels to numerical values
label_map = {'Positive': 1, 'Negative': 0}

# Use the map function to replace the labels with numerical values
train_df['Sepssis'] = train_df['Sepssis'].map(label_map)


In [142]:
# Initialize the scalers
minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

# Define the columns to scale (excluding the target column, if any)
columns_to_scale = ['Plasma glucose', 'Blood Work Result-1', 'Blood Pressure', 'Blood Work Result-2', 'Blood Work Result-3', 'Body mass index', 'Blood Work Result-4', 'Age', 'Insurance']

# Apply Min-Max Scaling
train_df[columns_to_scale] = minmax_scaler.fit_transform(train_df[columns_to_scale])
#test_df[columns_to_scale] = minmax_scaler.transform(test_df[columns_to_scale])

# Apply Standardization
# train_df[columns_to_scale] = standard_scaler.fit_transform(train_df[columns_to_scale])
# test_df[columns_to_scale] = standard_scaler.transform(test_df[columns_to_scale])

# Uncomment the lines above if you want to use Standardization instead of Min-Max Scaling

# Now train_df and test_df contain the scaled data


In [95]:
#Split train_df into features (X) and target (Y)
X = train_df.drop(columns='Sepssis')
Y = train_df['Sepssis']
X_train, X_unseen, Y_train, Y_unseen = train_test_split(X, Y, test_size=0.2, random_state=2)

In [96]:
#Create and train Logistic Regression model
logistic_model = LogisticRegression()
logistic_model.fit(X_train, Y_train)

LogisticRegression()

In [97]:
#Make predictions on the unseen data (X_unseen)
unseen_predictions = logistic_model.predict(X_unseen)

In [98]:
# Compute the accuracy on the unseen data
unseen_accuracy = accuracy_score(Y_unseen, unseen_predictions)
print("Accuracy on the unseen data:", unseen_accuracy)

Accuracy on the unseen data: 0.7833333333333333


In [99]:
#Make predictions on the test_df using the Logistic Regression model
test_predictions = logistic_model.predict(test_df)

In [100]:
#
# Assuming you want to add the predictions as a new column 'Prediction' in test_df
test_df['Prediction'] = test_predictions

# Optionally, you can save the test_df with predictions to a new CSV file
# test_df.to_csv('test_df_with_predictions.csv', index=False)

# Display the test_df with predictions
test_df

,Plasma glucose,Blood Work Result-1,Blood Pressure,Blood Work Result-2,Blood Work Result-3,Body mass index,Blood Work Result-4,Age,Insurance,Prediction
ID,,,,,,,,,,
ICU200609,1,109,38,18,120,23.1,0.407,26,1,1
ICU200610,1,108,88,19,0,27.1,0.400,24,1,1
ICU200611,6,96,0,0,0,23.7,0.190,28,1,1
ICU200612,1,124,74,36,0,27.8,0.100,30,1,1
ICU200613,7,150,78,29,126,35.2,0.692,54,0,1
...,...,...,...,...,...,...,...,...,...,...
ICU200773,10,101,76,48,180,32.9,0.171,63,1,1
ICU200774,2,122,70,27,0,36.8,0.340,27,1,1
ICU200775,5,121,72,23,112,26.2,0.245,30,1,1


In [108]:
test_df['Prediction'].value_counts()

1    169
Name: Prediction, dtype: int64

In [122]:
input_data = (0.176471,	0.570707,	0.409836,	0.101010,	0.100473,	0.439642,	0.233988,	0.066667,	1.0)

# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = logistic_model.predict(input_data_reshaped)
print(prediction)

if (prediction[0] == 0):
  print('The person is not not having Sepssis')
else:
  print('The person is has Sepssis')

[0]
The person is not not having Sepssis


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [124]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

In [131]:
# Create and train the Random Forest Classifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, Y_train)

RandomForestClassifier()

In [133]:
# Compute the accuracy on the unseen data
unseen_accuracy = accuracy_score(Y_unseen, unseen_predictions)
print("Accuracy on the unseen data:", unseen_accuracy)

Accuracy on the unseen data: 0.775


In [128]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.7578288100208769


In [129]:
# accuracy score on the test data
X_test_prediction = classifier.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [130]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.75
